<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/lecture-idea/15_optimization/015_two_dimensional_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
import numpy as np
import scipy.optimize as so



In [ ]:
# ref : https://matplotlib.org/stable/gallery/
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm



## 2차원 최적화<br>Two dimensional optimizations

다음과 같은 비용 함수를 생각해 보자.<br>Let's think about a cost function as follows.

$$
C(x_0, x_1) = \frac{x_0^2}{2^2} + \frac{x_1^2}{1^2}
$$

파이썬으로는 다음과 같이 구현할 수 있을 것이다.<br>We may implement in python as follows.



In [ ]:
def c(x:np.ndarray, a:float=2, b:float=1) -> float:
    x0 = x[0]
    x1 = x[1]
    
    return (x0 * x0) / (a * a) + (x1 * x1) / (b * b)



시각화 해 보자.<br>Let's visualize.



In [ ]:
def plot_cost():
    # ref : https://matplotlib.org/stable/gallery/

    fig = plt.figure(figsize=(15, 6))
    ax1 = plt.subplot(1, 2, 1)
    ax2 = plt.subplot(1, 2, 2, projection="3d")

    x = np.linspace(-4, 4)
    y = np.linspace(-2, 2)
    X, Y = np.meshgrid(x, y)

    Z = c((X, Y))

    cset = ax1.contour(X, Y, Z, cmap=cm.coolwarm)

    surf = ax2.plot_surface(X, Y, Z, antialiased=True, cmap=cm.viridis, alpha=0.5)
    fig.colorbar(surf)

    return ax1, ax2



In [ ]:
plot_cost()
plt.show()



중간 과정의 그래프를 그려 주는 비용 함수를 선언<br>Declare another cost function that will plot intermediate results



In [ ]:
def get_cost_with_plot(a=2, b=1, b_triangle=True):

    x0_history = []
    x1_history = []
    c_history = []

    def cost_with_plot(x, a=a, b=b):
        '''
        이런 함수를 클로져 라고 부름. 다른 함수의 내부 함수이면서 해당 함수의 반환값.
        This is a closuer; an internal function being a return value
        '''
        ax1, ax2 = plot_cost()

        result = c(x)

        x0_history.append(x[0])
        x1_history.append(x[1])
        c_history.append(result)

        ax1.plot(x0_history, x1_history, '.')
        ax2.plot(x0_history, x1_history, c_history, '.')

        if b_triangle and (3 <= len(x0_history)):
            ax1.plot(
                x0_history[-3:]+[x0_history[-3]],
                x1_history[-3:]+[x1_history[-3]],
                '-'
            )
            ax2.plot(
                x0_history[-3:]+[x0_history[-3]],
                x1_history[-3:]+[x1_history[-3]],
                c_history[-3:]+[c_history[-3]],
                '-'
            )

        plt.show()

        return result

    return cost_with_plot



In [ ]:
cost_with_plot = get_cost_with_plot()



### Nelder-Mead 법
ref : [[0]](https://en.wikipedia.org/wiki/Nelder-Mead_method)<br>
Nelder-Mead 법은 비용함수의 독립변수가 $n$ 차원인 경우, $n+1$ 개의 점으로 이루어진 **simplex**를 이용한다.<br>
If the independend variables of the cost function is $n$-dimensional, the Nelder-Mead method uses a **simplex** of $n+1$ vertices.



In [ ]:
nm_result = so.minimize(cost_with_plot, [3.0, 1.0], method="Nelder-Mead")



In [ ]:
nm_result



### Newton-CG 법
비용함수를 각각 $x_0$, $x_1$에 대해 편미분 해 보자.<br>Let's get the partial derivatives of the cost function over $x_0$ and $x_1$.
$$
C(x_0, x_1) = \frac{x_0^2}{2^2} + \frac{x_1^2}{1^2} \\
\frac{\partial C}{\partial x_0} = 2 \cdot \frac{x_0}{2^2} \\
\frac{\partial C}{\partial x_1} = 2 \cdot \frac{x_1}{1^2}
$$
파이썬으로는 다음과 같이 구현할 수 있을 것이다.<br>One may implement in python as follows.



In [ ]:
def jacobian(x, a=2, b=1):
    x0 = x[0]
    x1 = x[1]
    return np.array((2 * x0 / (a*a), 2 * x1 / (b*b),))



최적화에도 기울기를 사용할 수 있다.<br>We can also use the slopes in the optimization.



In [ ]:
cost_with_plot = get_cost_with_plot(b_triangle=False)



In [ ]:
fmin_newton = so.minimize(cost_with_plot, [3.0, 1.0], jac=jacobian, method="newton-cg")



In [ ]:
fmin_newton

